<a href="https://colab.research.google.com/github/zhiyingproject/tongueDiagnosis/blob/main/imgSeg.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [1]:
# This notebook is to train a model for locating tongues from people's faces. It is for tongue diagnosis using traditional Chinese Medical Theory (TCM)

!pip install --upgrade tensorflow==1.15.0
!pip install --upgrade tensorflow-gpu==1.15
!pip install --upgrade keras==2.2.4
!python -m pip install -U scikit-image==0.16.2

     |████████████████████████████████| 412.3 MB 19 kB/s 
     |████████████████████████████████| 503 kB 39.8 MB/s 
     |████████████████████████████████| 3.8 MB 37.1 MB/s 
     |████████████████████████████████| 50 kB 6.0 MB/s 
  Created wheel for gast: filename=gast-0.2.2-py3-none-any.whl size=7553 sha256=e7f5cb180eb344ac19bd32a721d036435d7f828eb0d6eb4dca4508cc3e3777bd
  Stored in directory: /root/.cache/pip/wheels/21/7f/02/420f32a803f7d0967b48dd823da3f558c5166991bfd204eef3
Successfully built gast
  Attempting uninstall: tensorflow-estimator
    Found existing installation: tensorflow-estimator 2.5.0
    Uninstalling tensorflow-estimator-2.5.0:
      Successfully uninstalled tensorflow-estimator-2.5.0
  Attempting uninstall: tensorboard
    Found existing installation: tensorboard 2.5.0
    Uninstalling tensorboard-2.5.0:
      Successfully uninstalled tensorboard-2.5.0
  Attempting uninstall: gast
    Found existing installation: gast 0.4.0
    Uninstalling gast-0.4.0:
      Succes

In [2]:
# Restarting the kernel to enable tensorflow v1.14
from IPython.core.display import HTML
HTML("<script>Jupyter.notebook.kernel.restart()</script>")

In [3]:
# Install Mask RCNN
!git clone https://github.com/matterport/Mask_RCNN.git

Cloning into 'Mask_RCNN'...
remote: Enumerating objects: 956, done.
remote: Total 956 (delta 0), reused 0 (delta 0), pack-reused 956
Receiving objects: 100% (956/956), 125.23 MiB | 34.26 MiB/s, done.
Resolving deltas: 100% (562/562), done.


In [ ]:
cd Mask_RCNN/

/content/Mask_RCNN


In [ ]:
# Setup libraries for Mas RCNN
!python setup.py install
!pip show mask-rcnn

/usr/local/lib/python3.7/dist-packages/setuptools/dist.py:700: UserWarning: Usage of dash-separated 'description-file' will not be supported in future versions. Please use the underscore name 'description_file' instead
  % (opt, underscore_opt))
/usr/local/lib/python3.7/dist-packages/setuptools/dist.py:700: UserWarning: Usage of dash-separated 'license-file' will not be supported in future versions. Please use the underscore name 'license_file' instead
  % (opt, underscore_opt))
/usr/local/lib/python3.7/dist-packages/setuptools/dist.py:700: UserWarning: Usage of dash-separated 'requirements-file' will not be supported in future versions. Please use the underscore name 'requirements_file' instead
  % (opt, underscore_opt))
running install
running bdist_egg
running egg_info
writing mask_rcnn.egg-info/PKG-INFO
writing dependency_links to mask_rcnn.egg-info/dependency_links.txt
writing top-level names to mask_rcnn.egg-info/top_level.txt
reading manifest template 'MANIFEST.in'
adding licens

In [ ]:
from google.colab import drive
drive.mount('/content/drive')

Drive already mounted at /content/drive; to attempt to forcibly remount, call drive.mount("/content/drive", force_remount=True).


In [ ]:
ds_folder = '/content/drive/MyDrive/backup/tongueDiagnosis/segTraining'
training_threshold = 140
model_path = '/content/drive/MyDrive/backup/tongueDiagnosis/models'

In [ ]:

import pathlib
import shutil
import cv2
import matplotlib.pyplot as plt
import xml.etree.ElementTree as ET
import sys
from mrcnn.utils import Dataset, extract_bboxes, compute_ap
from mrcnn.config import Config
from mrcnn.visualize import display_instances
from numpy import zeros, asarray
from tensorflow.keras.utils import plot_model
from mrcnn.model import MaskRCNN, load_image_gt, mold_image

Using TensorFlow backend.


In [ ]:
class tongueDataset(Dataset):
    def load_dataset(self, dataset_dir, is_train=True):
        self.add_class("dataset", 1, "tongue")

        images_dir = dataset_dir
        annotations_dir = f'{dataset_dir}/annots'
        image_files = pathlib.Path(dataset_dir).rglob('*.jpg')
        for ifile, filename in enumerate(list(image_files)):
            image_id = filename.stem
            if is_train and ifile >= training_threshold:
                continue
            if not is_train and ifile < training_threshold:
                continue
            img_path = filename
            ann_path = f'{annotations_dir}/{filename.stem}.xml'
            self.add_image('dataset', image_id=image_id, path=img_path, annotation=ann_path)

    def extract_boxes(self, filename):
        tree = ET.parse(filename)
        root = tree.getroot()
        boxes = list()
        for box in root.findall('.//bndbox'):
            xmin = int(box.find('xmin').text)
            ymin = int(box.find('ymin').text)
            xmax = int(box.find('xmax').text)
            ymax = int(box.find('ymax').text)
            coors = [xmin, ymin, xmax, ymax]
            boxes.append(coors)
        width = int(root.find('.//size/width').text)
        height = int(root.find('.//size/height').text)
        return boxes, width, height

    def load_mask(self, image_id):
        info = self.image_info[image_id]
        path = info['annotation']
        boxes, w, h = self.extract_boxes(path)
        masks = zeros([h, w, len(boxes)], dtype='uint8')

        class_ids = []
        for i in range(len(boxes)):
            box = boxes[i]
            row_s, row_e = box[1], box[3]
            col_s, col_e = box[0], box[2]
            masks[row_s:row_e, col_s:col_e, i] = 1
            class_ids.append(self.class_names.index('tongue'))
        return masks, asarray(class_ids, dtype='int32')

    def image_reference(self, image_id):
        info = self.image_info[image_id]
        return info['path']

In [ ]:
class tongueConfig(Config):
    NAME = 'tongue_cfg'
    NUM_CLASSES = 1 + 1
    STEPS_PER_EPOCH = training_threshold
    GPU_COUNT = 1
    IMAGES_PER_GPU = 1

In [ ]:
training_ds = tongueDataset()
training_ds.load_dataset(ds_folder, is_train=True)
training_ds.prepare()
print('Train: %d' % len(training_ds.image_ids))

test_ds = tongueDataset()
test_ds.load_dataset(ds_folder, is_train=False)
test_ds.prepare()
print('Test: %d' % len(test_ds.image_ids))

Train: 140
Test: 32


In [ ]:
config = tongueConfig()
config.display()

model = MaskRCNN(mode='training', model_dir=model_path, config=config)
model.load_weights('/content/drive/MyDrive/backup/tongueDiagnosis/models/mask_rcnn_coco.h5', 
                   by_name=True,
                   exclude=["mrcnn_class_logits", "mrcnn_bbox_fc",  "mrcnn_bbox", "mrcnn_mask"])
model.train(training_ds, test_ds, learning_rate=config.LEARNING_RATE, epochs=5, layers='heads')



Configurations:
BACKBONE                       resnet101
BACKBONE_STRIDES               [4, 8, 16, 32, 64]
BATCH_SIZE                     2
BBOX_STD_DEV                   [0.1 0.1 0.2 0.2]
COMPUTE_BACKBONE_SHAPE         None
DETECTION_MAX_INSTANCES        100
DETECTION_MIN_CONFIDENCE       0.7
DETECTION_NMS_THRESHOLD        0.3
FPN_CLASSIF_FC_LAYERS_SIZE     1024
GPU_COUNT                      1
GRADIENT_CLIP_NORM             5.0
IMAGES_PER_GPU                 2
IMAGE_CHANNEL_COUNT            3
IMAGE_MAX_DIM                  1024
IMAGE_META_SIZE                14
IMAGE_MIN_DIM                  800
IMAGE_MIN_SCALE                0
IMAGE_RESIZE_MODE              square
IMAGE_SHAPE                    [1024 1024    3]
LEARNING_MOMENTUM              0.9
LEARNING_RATE                  0.001
LOSS_WEIGHTS                   {'rpn_class_loss': 1.0, 'rpn_bbox_loss': 1.0, 'mrcnn_class_loss': 1.0, 'mrcnn_bbox_loss': 1.0, 'mrcnn_mask_loss': 1.0}
MASK_POOL_SIZE                 14
MASK_SHAPE         

/usr/local/lib/python3.7/dist-packages/tensorflow_core/python/framework/indexed_slices.py:424: UserWarning: Converting sparse IndexedSlices to a dense Tensor of unknown shape. This may consume a large amount of memory.
  "Converting sparse IndexedSlices to a dense Tensor of unknown shape. "
/usr/local/lib/python3.7/dist-packages/tensorflow_core/python/framework/indexed_slices.py:424: UserWarning: Converting sparse IndexedSlices to a dense Tensor of unknown shape. This may consume a large amount of memory.
  "Converting sparse IndexedSlices to a dense Tensor of unknown shape. "
/usr/local/lib/python3.7/dist-packages/tensorflow_core/python/framework/indexed_slices.py:424: UserWarning: Converting sparse IndexedSlices to a dense Tensor of unknown shape. This may consume a large amount of memory.
  "Converting sparse IndexedSlices to a dense Tensor of unknown shape. "


/usr/local/lib/python3.7/dist-packages/keras/engine/training_generator.py:47: UserWarning: Using a generator with `use_multiprocessing=True` and multiple workers may duplicate your data. Please consider using the`keras.utils.Sequence class.
  UserWarning('Using a generator with `use_multiprocessing=True`'




Epoch 1/5
140/140 [==============================] - 16596s 119s/step - loss: 0.8833 - rpn_class_loss: 0.0027 - rpn_bbox_loss: 0.1866 - mrcnn_class_loss: 0.0239 - mrcnn_bbox_loss: 0.3309 - mrcnn_mask_loss: 0.3390 - val_loss: 0.6406 - val_rpn_class_loss: 0.0040 - val_rpn_bbox_loss: 0.2062 - val_mrcnn_class_loss: 0.0190 - val_mrcnn_bbox_loss: 0.2024 - val_mrcnn_mask_loss: 0.2090

Epoch 2/5
 54/140 [==========>...................] - ETA: 2:31:33 - loss: 0.4236 - rpn_class_loss: 0.0024 - rpn_bbox_loss: 0.1169 - mrcnn_class_loss: 0.0159 - mrcnn_bbox_loss: 0.1232 - mrcnn_mask_loss: 0.1651